In [1]:
import pandas as pd
import numpy as np

from tokenizers import ByteLevelBPETokenizer

import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from transformers import TFBertModel,  BertConfig, BertTokenizer, AutoModel, PreTrainedTokenizerFast
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, LSTM, Flatten
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy
import pickle
import warnings

In [2]:
dataset = pd.read_csv('data_cleaned.csv',
                   converters={"preprocessedTags": lambda x: x.strip("[]").replace("'","").split(", ")})

dataset = dataset[['unstemmed_desc']].dropna()
dataset.head()

,unstemmed_desc
0,vs cakephp vs zend vs cakephp vs zend cakephp ...
1,tools generating mock data tools generating mo...
2,laravel use statement non name cache effect la...
3,add client authentication add client authentic...
4,variable error variable error system namespace...


In [3]:
dataset.to_csv(r'pandas.txt', header=None, index=None, sep=' ', mode='a')

In [4]:
path = "pandas.txt"

# Tokenizer from scratch

https://arxiv.org/abs/1508.07909 <br/>
https://huggingface.co/blog/how-to-train

In [5]:
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

In [10]:
tokenizer.train(files=path, vocab_size=52_000, min_frequency=50, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [11]:
tokenizer

Tokenizer(vocabulary_size=10570, model=ByteLevelBPE, add_prefix_space=False, lowercase=False, dropout=None, unicode_normalizer=None, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)

In [12]:
# Save the tokenizer you trained
tokenizer.save("byte-level-BPE.tokenizer.json")

# Load it using transformers
tokenizer = PreTrainedTokenizerFast(tokenizer_file="byte-level-BPE.tokenizer.json")

## Training a standard Neural Network

In [13]:
# Freeze the randomness
from numpy.random import seed
seed(1337)
import tensorflow as tf
tf.random.set_seed(42)

In [14]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [15]:
globalStrategy = "retrain" # Either retrain or keep

In [16]:
def scoring(data, predict):
    f1 = []
    jaccard = []
    threshold = []
    for i in np.arange(0.30, 0.99, 0.01):
        predict_ = np.where(predict >= i, 1, 0)
        f1_micro = metrics.f1_score(data, predict_, average = 'micro')
        jaccard_micro = metrics.jaccard_score(data, predict_, average = 'micro')
        f1.append(f1_micro)
        jaccard.append(jaccard_micro)
        threshold.append(i)
    
    results = pd.DataFrame()
    results['Threshold'] = threshold
    results['F1_micro'] = f1
    results['Jaccard_micro'] = jaccard
    results = results[results['F1_micro'] == results['F1_micro'].max()]
    return results

In [17]:
data = pd.read_csv('data_cleaned.csv',
                   converters={"preprocessedTags": lambda x: x.strip("[]").replace("'","").split(", ")})
#data = pd.read_csv('data_cleaned2.csv',
#                   converters={"preprocessedTags": lambda x: x.strip("[]").replace("'","").split(", ")})

data = data[['desc', 'unstemmed_desc', 'preprocessedTags', 'Tag1']].dropna()
data.head()

,desc,unstemmed_desc,preprocessedTags,Tag1
0,vs cakephp vs zend vs cakephp vs zend cakephp ...,vs cakephp vs zend vs cakephp vs zend cakephp ...,[php],php
1,tool generat mock data tool generat mock data ...,tools generating mock data tools generating mo...,[testing],testing
2,laravel use statement non name cach effect lar...,laravel use statement non name cache effect la...,"[php, laravel]",php
3,add client authent add client authent server r...,add client authentication add client authentic...,[java],java
4,variabl error variabl error system namespac cl...,variable error variable error system namespace...,[c#],c#


In [18]:
x = data['desc']
x_unstemmed = data['unstemmed_desc']
y = data['preprocessedTags']
# y_tag = data['Tag1']
mb = MultiLabelBinarizer()
y_encoded = mb.fit_transform(y)
# y_encoded = mb.fit_transform(y_tag)

In [19]:
# x_train, x_test split
x_train, x_test, y_train, y_test = train_test_split(x_unstemmed,
                                                    y_encoded,
                                                    test_size=0.2,
                                                    random_state=42)

In [20]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [21]:
# Ready output data for the model
max_length=250

x_train_toke = tokenizer(text=x_train.to_list(),
                         add_special_tokens=True,
                         max_length=max_length,
                         truncation=True,
                         padding=True, 
                         return_tensors='tf',
                         return_token_type_ids=False,
                         return_attention_mask=False,
                         verbose=True)

x_test_toke = tokenizer(text=x_test.to_list(),
                        add_special_tokens=True,
                        max_length=max_length,
                        truncation=True,
                        padding=True,
                        return_tensors='tf',
                        return_token_type_ids=False,
                        return_attention_mask=False,
                        verbose=True)

In [22]:
x_train_toke = x_train_toke['input_ids']
x_test_toke = x_test_toke['input_ids']

In [40]:
# Define the model
xInput = Input(shape=(max_length))
x_ = Dense(500, activation='relu')(xInput)
x_ = Dense(500, activation='relu')(x_)
output = Dense(len(y_encoded[0]), activation='sigmoid')(x_)

personalizedTokenizer = Model(inputs=xInput, outputs=output, name='Baseline')

# Compile the model
personalizedTokenizer.compile(loss=CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
                              optimizer=Adam(learning_rate=0.00000001),
                              metrics=['acc'])

In [41]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                            mode='min',
                                            restore_best_weights=True)

In [42]:
personalizedTokenizer.summary()

Model: "Baseline"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 250)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               125500    
_________________________________________________________________
dense_4 (Dense)              (None, 500)               250500    
_________________________________________________________________
dense_5 (Dense)              (None, 206)               103206    
Total params: 479,206
Trainable params: 479,206
Non-trainable params: 0
_________________________________________________________________


In [43]:
epochs = 200
batch_size = 128

In [44]:
# Load the baseline, if does not exist then train one
if globalStrategy == 'retrain' or globalStrategy == 'retrainPersonalizedTokenizer':
    epochs = epochs
    batch_size=batch_size
    history = personalizedTokenizer.fit(x_train_toke, y_train,
                                        epochs=epochs,
                                        validation_split=0.1,
                                        callbacks=[callback],
                                        verbose=1)

    personalizedTokenizer.save('/home/mlmaster/Code/Ing_ml_P7/personalizedTokenizer/')
    personalizedTokenizer = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/personalizedTokenizer/')
else:
    try:
        personalizedTokenizer = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/personalizedTokenizer/')
    except OSError:
        epochs = epochs
        batch_size=batch_size
        history = personalizedTokenizer.fit(x_train_toke, y_train,
                                            epochs=epochs,
                                            validation_split=0.1,
                                            callbacks=[callback],
                                            verbose=1)

        personalizedTokenizer.save('/home/mlmaster/Code/Ing_ml_P7/personalizedTokenizer/')
        personalizedTokenizer = tf.keras.models.load_model('/home/mlmaster/Code/Ing_ml_P7/personalizedTokenizer/')

Epoch 1/200
1059/1059 [==============================] - 2s 2ms/step - loss: 21767.8896 - acc: 7.1163e-04 - val_loss: 21802.8105 - val_acc: 0.0011
Epoch 2/200
1059/1059 [==============================] - 2s 2ms/step - loss: 21219.2358 - acc: 4.8994e-04 - val_loss: 21356.9316 - val_acc: 0.0013
Epoch 3/200
1059/1059 [==============================] - 2s 2ms/step - loss: 20590.5859 - acc: 7.1453e-04 - val_loss: 20958.9707 - val_acc: 0.0019
Epoch 4/200
1059/1059 [==============================] - 2s 2ms/step - loss: 20422.7196 - acc: 7.7984e-04 - val_loss: 20599.1855 - val_acc: 0.0024
Epoch 5/200
1059/1059 [==============================] - 2s 2ms/step - loss: 19898.9763 - acc: 7.4349e-04 - val_loss: 20268.3477 - val_acc: 0.0029
Epoch 6/200
1059/1059 [==============================] - 2s 2ms/step - loss: 19550.6301 - acc: 8.5584e-04 - val_loss: 19957.9922 - val_acc: 0.0027
Epoch 7/200
1059/1059 [==============================] - 2s 2ms/step - loss: 19308.2100 - acc: 6.9058e-04 - val_loss: 

Epoch 58/200
1059/1059 [==============================] - 2s 2ms/step - loss: 11346.1817 - acc: 0.0215 - val_loss: 11555.0244 - val_acc: 0.0205
Epoch 59/200
1059/1059 [==============================] - 2s 2ms/step - loss: 11182.9555 - acc: 0.0205 - val_loss: 11452.3994 - val_acc: 0.0210
Epoch 60/200
1059/1059 [==============================] - 2s 2ms/step - loss: 11128.2815 - acc: 0.0199 - val_loss: 11352.6924 - val_acc: 0.0210
Epoch 61/200
1059/1059 [==============================] - 2s 2ms/step - loss: 11035.4465 - acc: 0.0217 - val_loss: 11256.0566 - val_acc: 0.0218
Epoch 62/200
1059/1059 [==============================] - 2s 2ms/step - loss: 10888.1331 - acc: 0.0227 - val_loss: 11162.7910 - val_acc: 0.0218
Epoch 63/200
1059/1059 [==============================] - 2s 2ms/step - loss: 10923.3999 - acc: 0.0222 - val_loss: 11072.8672 - val_acc: 0.0234
Epoch 64/200
1059/1059 [==============================] - 2s 2ms/step - loss: 10795.0258 - acc: 0.0230 - val_loss: 10985.9443 - val_acc:

1059/1059 [==============================] - 2s 2ms/step - loss: 7865.5430 - acc: 0.0346 - val_loss: 7997.6504 - val_acc: 0.0335
Epoch 116/200
1059/1059 [==============================] - 2s 2ms/step - loss: 7856.7827 - acc: 0.0317 - val_loss: 7970.7607 - val_acc: 0.0340
Epoch 117/200
1059/1059 [==============================] - 2s 2ms/step - loss: 7842.4617 - acc: 0.0335 - val_loss: 7944.6318 - val_acc: 0.0345
Epoch 118/200
1059/1059 [==============================] - 2s 2ms/step - loss: 7801.3266 - acc: 0.0356 - val_loss: 7919.2061 - val_acc: 0.0348
Epoch 119/200
1059/1059 [==============================] - 2s 2ms/step - loss: 7777.9889 - acc: 0.0332 - val_loss: 7894.2764 - val_acc: 0.0351
Epoch 120/200
1059/1059 [==============================] - 2s 2ms/step - loss: 7745.9144 - acc: 0.0341 - val_loss: 7869.5938 - val_acc: 0.0353
Epoch 121/200
1059/1059 [==============================] - 2s 2ms/step - loss: 7762.7623 - acc: 0.0351 - val_loss: 7845.2603 - val_acc: 0.0353
Epoch 122/200

1059/1059 [==============================] - 2s 2ms/step - loss: 6794.7806 - acc: 0.0373 - val_loss: 6914.6367 - val_acc: 0.0382
Epoch 173/200
1059/1059 [==============================] - 2s 2ms/step - loss: 6851.7793 - acc: 0.0402 - val_loss: 6901.4683 - val_acc: 0.0380
Epoch 174/200
1059/1059 [==============================] - 2s 2ms/step - loss: 6798.2099 - acc: 0.0401 - val_loss: 6888.1694 - val_acc: 0.0380
Epoch 175/200
1059/1059 [==============================] - 2s 2ms/step - loss: 6847.5254 - acc: 0.0377 - val_loss: 6875.2231 - val_acc: 0.0380
Epoch 176/200
1059/1059 [==============================] - 2s 2ms/step - loss: 6719.4333 - acc: 0.0404 - val_loss: 6862.1309 - val_acc: 0.0380
Epoch 177/200
1059/1059 [==============================] - 2s 2ms/step - loss: 6753.3096 - acc: 0.0391 - val_loss: 6849.2266 - val_acc: 0.0380
Epoch 178/200
1059/1059 [==============================] - 2s 2ms/step - loss: 6739.2854 - acc: 0.0406 - val_loss: 6836.3589 - val_acc: 0.0377
Epoch 179/200

In [45]:
personalizedTokenizer.evaluate(x_test_toke, y_test, verbose=1)

295/295 [==============================] - 0s 1ms/step - loss: 6455.0566 - acc: 0.0426


[6455.056640625, 0.04260971024632454]

In [46]:
predict = personalizedTokenizer.predict(x_test_toke)

In [47]:
#predict_ = np.where(predict>0.55, 1, 0)
scoring(y_test, predict)

,Threshold,F1_micro,Jaccard_micro
68,0.98,0.024289,0.012294


In [48]:
predict[12]

array([1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 1.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
       0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e

In [49]:
print(x_train[15])
print(tokenizer.tokenize(x_train[15]))
print(x_train_toke[15])

read msg files read msg files need read outlook msg file net without com api outlook cos installed machines app run free 3rd party libraries want extract cc fields sent receive date fields would good also stored msg files
['read', 'Ġmsg', 'Ġfiles', 'Ġread', 'Ġmsg', 'Ġfiles', 'Ġneed', 'Ġread', 'Ġoutlook', 'Ġmsg', 'Ġfile', 'Ġnet', 'Ġwithout', 'Ġcom', 'Ġapi', 'Ġoutlook', 'Ġcos', 'Ġinstalled', 'Ġmachines', 'Ġapp', 'Ġrun', 'Ġfree', 'Ġ3', 'rd', 'Ġparty', 'Ġlibraries', 'Ġwant', 'Ġextract', 'Ġcc', 'Ġfields', 'Ġsent', 'Ġreceive', 'Ġdate', 'Ġfields', 'Ġwould', 'Ġgood', 'Ġalso', 'Ġstored', 'Ġmsg', 'Ġfiles']
tf.Tensor(
[ 9898  1452  1607   471   471   563  3472  1452  1607   471   471   563
   976  8879  1607  1874   563  2202   559   418   450  1452  1607   471
   471  2030  1607   577  1188  1890   877   290   526   290   526   290
   526  1047  8495   290  2008  2008   450  1105   575  1607   675   729
  1105  1105  1105   675   675   729   408  1105  1607   675   408  5378
  1024  1607   675  

# Retrain Partially a Tokenizer